# Calculate SPS for species lookup tables 

### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os
from deep_translator import GoogleTranslator

<a id='esri'></a>
### Connect to ESRI

In [2]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [3]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


<a id='utils'></a>
### Utils

In [5]:
# To convert an existing hosted layer in an spatial dataframe
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

## Get lookups
These are species lookup tables published by MOL and used in production

In [7]:
amphibians = getHTfromId('c221a727e12048b2a6ec8e762bc5f478')
birds = getHTfromId('bcb31fd9091446a0af3cfdaed334a8da')
mammals = getHTfromId('212a3dd4665845deb5d2adf5b597aae0')
reptiles = getHTfromId('5b606a03b3fc431e8d4b9191c88bc2b9')

In [9]:
amphibians.head(1)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,ObjectId
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,0,None,1


In [20]:
amphibians['SPS'] = (amphibians['percent_protected']/amphibians['conservation_target'])*100
amphibians['SPS'] = amphibians['SPS'].astype(int)
amphibians['SPS'] = np.where(amphibians.SPS >100,100, amphibians.SPS)
amphibians.head(10)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,ObjectId,SPS
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,0,None,1,100
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,0,None,2,100
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,10,1,Northern Cricket Frog,3,33
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,10,1,"Southern Cricket Frog, Florida Cricket Frog (...",4,33
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,0,None,5,100
5,Adelophryne_adiastola,6,Adelophryne adiastola,201734,53455,26,18,0,0,0,None,6,100
6,Adelophryne_baturitensis,7,Adelophryne baturitensis,92,0,0,100,0,100,0,None,7,0
7,Adelophryne_gutturosa,8,Adelophryne gutturosa,232546,129334,56,16,0,0,0,None,8,100
8,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,0,1,None,9,100
9,Adelophryne_maranguapensis,9,Adelophryne maranguapensis,29,0,0,100,0,100,0,None,10,0


In [21]:
birds['SPS'] = (birds['percent_protected']/birds['conservation_target'])*100
birds['SPS'] = birds['SPS'].astype(int)
birds['SPS'] = np.where(birds.SPS >100,100, birds.SPS)
birds.head(10)


,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,ObjectId,SPS
0,Alectoris_rufa,301,Alectoris rufa,572449,127220,22,15,0,0,1,Red-legged Partridge,1,100
1,Acrocephalus_bistrigiceps,101,Acrocephalus bistrigiceps,1513200,38076,3,15,0,12,1,"Black-browed Reed Warbler, Black-browed Reed-W...",2,20
2,Aegithalos_iouschistos,151,Aegithalos iouschistos,221256,11812,5,17,0,12,1,"Black-browed Tit, Rufous-fronted Bushtit",3,29
3,Agapornis_lilianae,201,Agapornis lilianae,207531,84448,41,18,0,0,1,Lilian's Lovebird,4,100
4,Abeillia_abeillei,1,Abeillia abeillei,73403,11203,15,34,0,19,1,Emerald-chinned Hummingbird,5,44
5,Alectroenas_madagascariensis,302,Alectroenas madagascariensis,271875,26620,10,15,0,5,1,"Madagascar Blue-Pigeon, Madagascar Blue Pigeon",6,66
6,Acrocephalus_brevipennis,102,Acrocephalus brevipennis,2609,83,3,85,0,82,1,"Cape Verde Swamp Warbler, Cape Verde Warbler, ...",7,3
7,Aegithalos_leucogenys,152,Aegithalos leucogenys,109452,5659,5,28,0,23,1,"White-cheeked Tit, White-cheeked Bushtit",8,17
8,Agapornis_nigrigenis,202,Agapornis nigrigenis,42566,12851,30,42,0,12,1,Black-cheeked Lovebird,9,71
9,Abroscopus_albogularis,2,Abroscopus albogularis,1653436,81123,5,15,0,10,1,Rufous-faced Warbler,10,33


In [22]:
mammals['SPS'] = (mammals['percent_protected']/mammals['conservation_target'])*100
mammals['SPS'] = mammals['SPS'].astype(int)
mammals['SPS'] = np.where(mammals.SPS >100,100, mammals.SPS)
mammals.head(10)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,ObjectId,SPS
0,Abditomys_latidens,1,Abditomys latidens,2850,212,7,84,0,77,0,Luzon Broad-toothed Rat,1,8
1,Abeomelomys_sevia,2,Abeomelomys sevia,19298,1058,5,54,0,49,0,Papuan Abeomelomys,2,9
2,Abrawayaomys_chebezi,3,Abrawayaomys chebezi,110904,6791,6,28,0,22,0,None,3,21
3,Abrawayaomys_ruschii,4,Abrawayaomys ruschii,40824,22644,55,43,0,0,0,Ruschi’s Rat,4,100
4,Abrocoma_bennettii,5,Abrocoma bennettii,64370,1624,3,36,0,33,1,Bennett’s Chinchilla Rat,5,8
5,Abrocoma_boliviensis,6,Abrocoma boliviensis,708,704,99,100,0,1,0,Bolivian Chinchilla Rat,6,99
6,Abrocoma_budini,7,Abrocoma budini,396,0,0,100,0,100,0,Budin’s Chinchilla Rat,7,0
7,Abrocoma_cinerea,8,Abrocoma cinerea,172909,36700,21,21,0,0,0,Ashy Chinchilla Rat,8,100
8,Abrocoma_famatina,9,Abrocoma famatina,7615,1333,18,69,0,51,0,Famatina Chinchilla Rat,9,26
9,Abrocoma_schistacea,10,Abrocoma schistacea,15179,6484,43,58,0,15,0,None,10,74


In [23]:
reptiles['SPS'] = (reptiles['percent_protected']/reptiles['conservation_target'])*100
reptiles['SPS'] = reptiles['SPS'].astype(int)
reptiles['SPS'] = np.where(reptiles.SPS >100,100, reptiles.SPS)
reptiles.head(10)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,ObjectId,SPS
0,Ablepharus_bivittatus,1,Ablepharus bivittatus,154913,12860,8,22,0,14,1,"Twin-striped Skink, Two-Streaked Snake-Eyed Sk...",1,36
1,Ablepharus_budaki,2,Ablepharus budaki,104226,3619,3,28,0,25,0,Budak’s Snake-eyed Skink,2,10
2,Ablepharus_chernovi,3,Ablepharus chernovi,111197,833,1,27,0,26,0,Chernov’s Skink,3,3
3,Ablepharus_darvazi,4,Ablepharus darvazi,2736,126,5,85,0,80,0,None,4,5
4,Ablepharus_deserti,5,Ablepharus deserti,185850,14279,8,20,0,12,0,Desert Lidless Skink,5,40
5,Ablepharus_grayanus,6,Ablepharus grayanus,86712,5590,6,31,0,25,0,Minor Snake-eyed Skink,6,19
6,Ablepharus_kitaibelii,7,Ablepharus kitaibelii,612789,99911,16,15,0,0,1,"Juniper Skink, Snake-eyed skink\n, Johanniseid...",7,100
7,Ablepharus_lindbergi,8,Ablepharus lindbergi,12768,919,7,61,0,54,0,"Lindberg’s Twin-striped Skink, Lindberg’s Snak...",8,11
8,Ablepharus_pannonicus,9,Ablepharus pannonicus,2269086,125784,6,15,0,9,0,"Asian Snake-eyed Skink\n, Asiatisches Natterauge",9,40
9,Ablepharus_rueppellii,10,Ablepharus rueppellii,27230,1672,6,49,0,43,1,"Rueppel’s Snake-eyed skink\n, Rueppels Johanni...",10,12


In [24]:
# Save new tables
amphibians.to_csv('/Users/sofia/Documents/HE_Data/Lookup_Tables/amphibians_with_SPS.csv')
birds.to_csv('/Users/sofia/Documents/HE_Data/Lookup_Tables/birds_with_SPS.csv')
mammals.to_csv('/Users/sofia/Documents/HE_Data/Lookup_Tables/mammals_with_SPS.csv')
reptiles.to_csv('/Users/sofia/Documents/HE_Data/Lookup_Tables/reptiles_with_SPS.csv')

Publish new tables in AGOL to be tested in staging. These tables contain private data, so we also need to create whitelisted services. Once they have been tested they can be sent to production and the previous ones can be set as deprecated.